In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import glob 

In [2]:
print(os.getcwd())
%cd ../../..

/Users/elizagoler/Documents/RecessionFertility/source/derived/county_gfrs
/Users/elizagoler/Documents/RecessionFertility


In [3]:
parquet_files = glob.glob("datastore/output/nvss/processed/*.parquet")
# Sort the files numerically by year extracted from the filename
parquet_files_sorted = sorted(parquet_files, key=lambda f: int(os.path.splitext(os.path.basename(f))[0]))
dfs = []
for f in parquet_files_sorted:
    year = os.path.splitext(os.path.basename(f))[0]
    df = pd.read_parquet(f)
    df['datayear'] = year
    dfs.append(df)
all_years_df = pd.concat(dfs, ignore_index=True)

In [4]:
all_years_df.head()

,datayear,stateres,cntyres,recwt,cntyrfip,ocntyfips
0,1968,01,01020,NaN,NaN,NaN
1,1968,01,01001,NaN,NaN,NaN
2,1968,01,01001,NaN,NaN,NaN
3,1968,01,01026,NaN,NaN,NaN
4,1968,01,01001,NaN,NaN,NaN


In [5]:
np.sort(all_years_df['datayear'].unique())

array(['1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975',
       '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983',
       '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003'], dtype=object)

In [6]:
years_all_nan_cntyrfip = all_years_df.groupby('datayear')['cntyrfip'].apply(lambda x: x.isna().all())
datayears_with_all_nan_cntyrfip = years_all_nan_cntyrfip[years_all_nan_cntyrfip].index.tolist()
print(f"Datayears with all cntyrfip NaN: {datayears_with_all_nan_cntyrfip}")


Datayears with all cntyrfip NaN: ['1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '2003']


** FIPS uses 999 to indicate unknown counties so 

In [ ]:
num_cntyres_ending_999 = all_years_df['cntyres'].astype(str).str.endswith('999').sum()
num_unique_cntyres_ending_999 = all_years_df.loc[all_years_df['cntyres'].astype(str).str.endswith('999'), 'cntyres'].nunique()
print(f"Number of unique cntyres values ending in '999': {num_unique_cntyres_ending_999}")

Number of unique cntyres values ending in '999': 50


In [43]:
# Show example rows where both 'cntyres' and 'cntyrfip' are populated (not null)
mask = all_years_df['cntyres'].notna() & all_years_df['cntyrfip'].notna()
both_populated = all_years_df[mask]

# Display about 70 example rows
example_rows = both_populated[['datayear', 'stateres', 'cntyres', 'cntyrfip']].head(70)
print("Example rows with both cntyres and cntyrfip populated (showing up to 70 rows):")
print(example_rows)


Example rows with both cntyres and cntyrfip populated (showing up to 70 rows):
         datayear stateres cntyres cntyrfip
32967773     1982       01   01001    01001
32967774     1982       01   01002    01003
32967775     1982       01   01002    01003
32967776     1982       01   01002    01003
32967777     1982       01   01002    01003
...           ...      ...     ...      ...
32967838     1982       01   01008    01015
32967839     1982       01   01008    01015
32967840     1982       01   01008    01015
32967841     1982       01   01008    01015
32967842     1982       01   01008    01015

[70 rows x 4 columns]


In [4]:
nchs_fips_crosswalk = pd.read_csv('datastore/raw/crosswalks/nchs_fips_crosswalk.csv')

In [5]:
nchs_fips_crosswalk.head()

,nchs_state,stateoc,nchs_county,countyoc,nchs_msa,metro,fipsst,fips_msa,countyname,statename,fipsco
0,1,1,1,1,196,1,1,5240,Autauga,Alabama,1001
1,1,1,2,2,192,1,1,5160,Baldwin,Alabama,1003
2,1,1,3,3,0,2,1,0,Barbour,Alabama,1005
3,1,1,4,4,0,2,1,0,Bibb,Alabama,1007
4,1,1,5,5,37,1,1,1000,Blount,Alabama,1009


In [6]:
nchs_fips_crosswalk = nchs_fips_crosswalk.rename(columns={
    'nchs_state': 'stateres',
    'nchs_county': 'cntyres'
})

In [7]:
nchs_fips_crosswalk['stateres'] = nchs_fips_crosswalk['stateres'].astype(str)
nchs_fips_crosswalk['cntyres'] = nchs_fips_crosswalk['cntyres'].astype(str)
nchs_fips_crosswalk.dtypes

stateres      object
stateoc        int64
cntyres       object
countyoc       int64
nchs_msa       int64
metro          int64
fipsst         int64
fips_msa       int64
countyname    object
statename     object
fipsco         int64
dtype: object

In [11]:
all_years_df.dtypes

datayear     object
stateres     object
cntyres      object
recwt        object
cntyrfip     object
ocntyfips    object
dtype: object

In [12]:
nchs_fips_crosswalk.head()

,stateres,stateoc,cntyres,countyoc,nchs_msa,metro,fipsst,fips_msa,countyname,statename,fipsco
0,1,1,1,1,196,1,1,5240,Autauga,Alabama,1001
1,1,1,2,2,192,1,1,5160,Baldwin,Alabama,1003
2,1,1,3,3,0,2,1,0,Barbour,Alabama,1005
3,1,1,4,4,0,2,1,0,Bibb,Alabama,1007
4,1,1,5,5,37,1,1,1000,Blount,Alabama,1009


In [13]:
all_years_df.head()

,datayear,stateres,cntyres,recwt,cntyrfip,ocntyfips
0,1968,01,01020,NaN,NaN,NaN
1,1968,01,01001,NaN,NaN,NaN
2,1968,01,01001,NaN,NaN,NaN
3,1968,01,01026,NaN,NaN,NaN
4,1968,01,01001,NaN,NaN,NaN


**Get the NCHS data in the right format for the crosswalk**

In [8]:
# Remove leading zeros from stateres column
all_years_df['stateres'] = all_years_df['stateres'].astype(str).str.lstrip('0')

# For cntyres: keep only last 3 digits, remove leading zeros
all_years_df['cntyres'] = all_years_df['cntyres'].astype(str).str[-3:].str.lstrip('0')

all_years_df.head()

,datayear,stateres,cntyres,recwt,cntyrfip,ocntyfips
0,1968,1,20,NaN,NaN,NaN
1,1968,1,1,NaN,NaN,NaN
2,1968,1,1,NaN,NaN,NaN
3,1968,1,26,NaN,NaN,NaN
4,1968,1,1,NaN,NaN,NaN


In [9]:
merged_df = all_years_df.merge(
    nchs_fips_crosswalk[['stateres', 'cntyres', 'fipsco']],
    on=['stateres', 'cntyres'],
    how='left'
)


In [16]:
merged_df.head()

,datayear,stateres,cntyres,recwt,cntyrfip,ocntyfips,fipsco
0,1968,1,20,NaN,NaN,NaN,1039.0
1,1968,1,1,NaN,NaN,NaN,1001.0
2,1968,1,1,NaN,NaN,NaN,1001.0
3,1968,1,26,NaN,NaN,NaN,1051.0
4,1968,1,1,NaN,NaN,NaN,1001.0


In [10]:
# Remove decimal, pad to five with leading zero if needed, and convert to string
merged_df['fipsco'] = merged_df['fipsco'].apply(
    lambda x: str(int(x)).zfill(5) if not pd.isna(x) else x
)
merged_df['fipsco'] = merged_df['fipsco'].astype(str)

## Tests to see if my answers are reasonable ##

In [28]:
nan_fipsco = merged_df['fipsco'].isna()
cntyres_999 = merged_df['cntyres'].astype(str).str.zfill(3).str[-3:] == '999'
share_999_in_nan_fipsco = (cntyres_999 & nan_fipsco).sum() / nan_fipsco.sum() if nan_fipsco.sum() else float('nan')
print(f"Share of rows where fipsco is NaN that also have cntyres = 999: {share_999_in_nan_fipsco:.3f}")


Share of rows where fipsco is NaN that also have cntyres = 999: 0.775


In [33]:
# Print some rows where cntyrfip is NOT NaN and fipsco is NOT NaN
print(merged_df[merged_df['cntyrfip'].notna() & merged_df['fipsco'].notna()].head())


         datayear stateres cntyres recwt cntyrfip ocntyfips fipsco
32967773     1982        1       1     1    01001       NaN  01001
32967774     1982        1       2     1    01003       NaN  01003
32967775     1982        1       2     1    01003       NaN  01003
32967776     1982        1       2     1    01003       NaN  01003
32967777     1982        1       2     1    01003       NaN  01003


In [39]:
# Get rows where 'cntyrfip' is not NaN and 'fipsco' is not 'nan' (as a string)
mask = merged_df['cntyrfip'].notna() & (merged_df['fipsco'] != 'nan')
subset = merged_df[mask]

# Compute the share where the two column values are NOT equal
n_total = len(subset)
n_not_equal = (subset['cntyrfip'] != subset['fipsco']).sum()
share_not_equal = n_not_equal / n_total if n_total > 0 else float('nan')

print(f"Share of rows (with non-NaN cntyrfip and fipsco!='nan') where the values are NOT equal: {share_not_equal:.3f}")


Share of rows (with non-NaN cntyrfip and fipsco!='nan') where the values are NOT equal: 0.048


In [11]:
# Print a sample of rows where cntyrfip and fipsco are both not NaN, but their values are NOT equal
mismatch_rows = merged_df[
    merged_df['cntyrfip'].notna() &
    (merged_df['fipsco'] != 'nan') &
    (merged_df['cntyrfip'] != merged_df['fipsco'])
]
print(mismatch_rows.head())


         datayear stateres cntyres recwt cntyrfip ocntyfips fipsco
33036103     1982       33      29     1    36061       NaN  00036
33036104     1982       33      29     1    36061       NaN  00036
33078563     1982       33      29     1    36081       NaN  00036
33097810     1982       33      29     1    36061       NaN  00036
33233011     1982       33      29     2    36005       NaN  00036


In [43]:
n_unique_triplets = mismatch_rows.drop_duplicates(subset=['datayear', 'stateres', 'cntyres']).shape[0]
print(f"Number of unique (datayear, stateres, cntyres) in mismatch_rows: {n_unique_triplets}")

Number of unique (datayear, stateres, cntyres) in mismatch_rows: 66


**Only 0.2% of state x county pairs with a cntyrfip in the original df did not match the fipsco brought in from the crosswalk merge**

In [12]:
unique_stateres_cntyres = mismatch_rows.drop_duplicates(subset=['stateres', 'cntyres'])[['stateres', 'cntyres']]
n_unique_stateres_cntyres = unique_stateres_cntyres.shape[0]
print(f"Number of unique (stateres, cntyres) pairs in mismatch_rows: {n_unique_stateres_cntyres}")
print("These pairs are:")
print(unique_stateres_cntyres)

Number of unique (stateres, cntyres) pairs in mismatch_rows: 6
These pairs are:
         stateres cntyres
33036103       33      29
78699833        3       8
78754260        2       3
78759952        3      12
78764887        3      14
78764946        3      11


In [13]:
# Total number of unique (stateres, cntyres) pairs in whole merged_df (where cntyrfip and fipsco not nan)
total_unique_pairs = merged_df[
    merged_df['cntyrfip'].notna() & (merged_df['fipsco'] != 'nan')
][['stateres', 'cntyres']].drop_duplicates()
n_total_pairs = total_unique_pairs.shape[0]

# Number of unique mismatched pairs (already computed above: n_unique_stateres_cntyres)
print(f"{n_unique_stateres_cntyres} out of {n_total_pairs} unique (stateres, cntyres) pairs had a mismatch between cntyrfip and fipsco.")
if n_total_pairs > 0:
    share_mismatched = n_unique_stateres_cntyres / n_total_pairs
else:
    share_mismatched = float('nan')
print(f"Share with mismatch: {share_mismatched:.3f}")

6 out of 3132 unique (stateres, cntyres) pairs had a mismatch between cntyrfip and fipsco.
Share with mismatch: 0.002


**For now I'm just dropping those counties (there are 6 of them)**

In [14]:
# Create a DataFrame that drops rows corresponding to the mismatched (stateres, cntyres) pairs
# First, create a set of tuples representing the mismatched pairs
mismatched_pairs = set(tuple(x) for x in unique_stateres_cntyres.values)

# Then filter merged_df to only keep rows **not** in those pairs
birth_records_w_fips = merged_df[~merged_df.set_index(['stateres', 'cntyres']).index.isin(mismatched_pairs)].copy()


In [49]:
print(int((len(merged_df) - len(mismatch_rows)) == len(birth_records_w_fips)))


0


In [15]:
birth_records_w_fips.head()

,datayear,stateres,cntyres,recwt,cntyrfip,ocntyfips,fipsco
0,1968,1,20,NaN,NaN,NaN,01039
1,1968,1,1,NaN,NaN,NaN,01001
2,1968,1,1,NaN,NaN,NaN,01001
3,1968,1,26,NaN,NaN,NaN,01051
4,1968,1,1,NaN,NaN,NaN,01001


In [17]:
# Assuming 'datastore/output/nvss' directory exists, if not, create it
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "nvss_records_w_fips.parquet")

# Write the DataFrame 'nvss_records_w_fips' to parquet
birth_records_w_fips.to_parquet(output_path, index=False)


## To understand: There are no counties with missing cntyrfip for years 1990-1994 even though documentation says these should only contain geographic info for counties of 100,000 or more ##

In [53]:
# Check for rows with cntyrfip missing (NaN or NA) for years 1990-1994
yrs_90_94 = [str(y) for y in range(1990, 1995)]
mask_yrs = birth_records_w_fips['datayear'].isin(yrs_90_94)
# Treat both actual NaN and string 'NA' or 'nan' as missing
mask_missing_cntyrfip = birth_records_w_fips['cntyrfip'].isna() | (birth_records_w_fips['cntyrfip'].astype(str).str.lower().isin(['nan', 'na']))
missing_90_94 = birth_records_w_fips[mask_yrs & mask_missing_cntyrfip]

print(f"Number of rows with missing cntyrfip for years 1990-1994: {len(missing_90_94)}")
if len(missing_90_94) > 0:
    print(missing_90_94.head(10))
else:
    print("No rows with missing cntyrfip for years 1990-1994.")


Number of rows with missing cntyrfip for years 1990-1994: 0
No rows with missing cntyrfip for years 1990-1994.


## 

## Now that we have a good pass at fips codes, making sure recwt column is right ##

From the documentation:
- Data for 1968-1971 are based on a 50% sample
- 

In [3]:
birth_records_w_fips = pd.read_parquet("datastore/output/nvss/nvss_records_w_fips.parquet")

In [4]:
# Find years where 'recwt' is missing for all rows
years = birth_records_w_fips['datayear'].unique()
years_with_all_recwt_missing = []
for year in years:
    df_year = birth_records_w_fips[birth_records_w_fips['datayear'] == year]
    # Treat both NaN and string 'NA'/'nan' as missing
    recwt_missing = df_year['recwt'].isna() | (df_year['recwt'].astype(str).str.lower().isin(['nan', 'na']))
    if recwt_missing.all():
        years_with_all_recwt_missing.append(year)
print("Years with 'recwt' missing for all rows:", years_with_all_recwt_missing)

Years with 'recwt' missing for all rows: ['1968', '1969', '1970', '1971']


In [5]:
for year in ['1968', '1969', '1970', '1971']:
    mask = birth_records_w_fips['datayear'] == year
    birth_records_w_fips.loc[mask, 'recwt'] = 2


In [6]:
# Create a new DataFrame that aggregates the number of births by datayear, stateres, cntyres, fipsco
# using recwt as the weight (defaulting to 1 if missing)
births_df = birth_records_w_fips.copy()
# recwt might be nan or missing, so default to 1
births_df['recwt'] = pd.to_numeric(births_df['recwt'], errors='coerce').fillna(1)
births_df['recwt'] = births_df['recwt'].astype(float)
agg_births_by_county_year = (
    births_df
    .groupby(['datayear', 'stateres', 'cntyres', 'fipsco'], dropna=False, as_index=False)
    .agg(births=('recwt', 'sum'))
)

In [9]:
agg_births_by_county_year.head()

,datayear,stateres,cntyres,fipsco,births
0,1968,1,1,01001,488
1,1968,1,10,01019,226
2,1968,1,11,01021,434
3,1968,1,12,01023,360
4,1968,1,13,01025,566


In [8]:
agg_births_by_county_year['births'] = agg_births_by_county_year['births'].astype(int)


In [10]:
agg_births_by_county_year.to_parquet("datastore/output/nvss/agg_births_by_county_year.parquet", index=False)